In [1]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/naver_comment.csv', encoding='utf8')
df.head()

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,엑스트라 데이즈,9.62,101,6,아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑...,126,0,42,버키 왜 하나만 블러예욬ㅋㅋㅋㅋㅋ\n이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다\...,9.65
1,원수를 사랑하라,9.95,31400,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129323,3789,984,그림체 마음에 드네 \n그림체 미친\n면접관중에 존나 보기싫은얼굴이보이는데?\n누가...,9.92
2,우투리,8.50,159,4,"학원액션물 만화계의 거장 임재원님의 2번째 작품 우투리입니다.... 스토리, 그림체...",194,5,51,진짜 퀄리티 말도 안되는데 덴마 이후로 미완결 작품은 그냥 혐오스럽다\n우와아아아!...,9.20
3,롭플롭,9.78,290,5,"나 이 작가 작품 좋아하네.... 잭슨의 관, 데빌샷 보고 바로 들어옴 \n잭슨의 ...",410,2,64,작가님 어케 이렇게 그천이신거에요\n노랑머리 친구 명암때문인진 몰라도장례식 이후에 ...,9.91
4,약초마을 연쇄살초사건,9.73,10226,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,62851,254,1463,ㅋㅋㅋㅋㅋㅋㅋㅋ\n조낸 무서운 이야기인데 약초란 사실과 그림체 때문에 잔혹함이 전달...,9.87


In [5]:
def train_to_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    
    lgbm = LGBMRegressor()
    xgb = XGBRegressor()
    rf = RandomForestRegressor()

    regs = [lgbm, xgb, rf]

    result = []
    for reg in regs:
        reg.fit(X_train, y_train)
        pred = reg.predict(X_val)
        mae = mean_absolute_error(y_val, pred)
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        print(reg.__class__.__name__)
        print(f'MAE : {mae:.4f}, RMSE : {rmse:.4f}')
        
    return regs

# 1. 텍스트 제외 모든 feature

In [6]:
X = df.drop(columns=['title', 'best', 'comment', 'star'])
y = df['star']

regs1 = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.1773, RMSE : 0.3286
XGBRegressor
MAE : 0.1891, RMSE : 0.4106
RandomForestRegressor
MAE : 0.1692, RMSE : 0.3719


# 2. 베댓 추천 수, 비추천 수 제외

In [7]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'best_recomm', 'best_unrecomm'])
y = df['star']

regs2 = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.1567, RMSE : 0.2952
XGBRegressor
MAE : 0.1860, RMSE : 0.4040
RandomForestRegressor
MAE : 0.1697, RMSE : 0.3830


## 추가적으로 베댓 수도 제외

In [8]:
X = X.drop(columns=['best_num'])

regs2_a = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.1560, RMSE : 0.2900
XGBRegressor
MAE : 0.1837, RMSE : 0.3937
RandomForestRegressor
MAE : 0.1677, RMSE : 0.3792


## 베댓 수만 제외

In [11]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'best_num'])
y = df['star']

regs2_b = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.1776, RMSE : 0.3320
XGBRegressor
MAE : 0.1982, RMSE : 0.4265
RandomForestRegressor
MAE : 0.1688, RMSE : 0.3752


# 3. 1화 평점 제외

In [9]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'score'])
y = df['star']

regs3 = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.3304, RMSE : 0.5263
XGBRegressor
MAE : 0.2946, RMSE : 0.5495
RandomForestRegressor
MAE : 0.2657, RMSE : 0.5126


# 4. 1화 평점, 평가 인원으로만

In [10]:
X = df[['score', 'score_num']]
y = df['star']

regs4 = train_to_evaluate(X, y)

LGBMRegressor
MAE : 0.1446, RMSE : 0.2774
XGBRegressor
MAE : 0.1901, RMSE : 0.4133
RandomForestRegressor
MAE : 0.1727, RMSE : 0.3801


당연한 결과겠지만, 1화 평점이 평균 평점을 예측하는 데 있어 매우 중요한 요소이며 그렇다고 1화 평점만으로는 평균 평점을 예측하는 데 무리가 있다.  
그리고 베댓 수, 베댓 추천 수, 베댓 비추천 수는 오히려 오차를 늘리는 요소이므로 향후 최종 모델에서는 제외하는 것으로...  
또한 TF-IDF 결과보다 더 우수한 결과가 나왔다.